In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os 

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/home/ccorbin/.config/gcloud/application_default_credentials.json' 
os.environ['GCLOUD_PROJECT'] = 'mining-clinical-decisions' 
%load_ext google.cloud.bigquery

from google.cloud import bigquery
client=bigquery.Client()

/share/sw/open/anaconda/3/lib/python3.6/site-packages/google/auth/_default.py:66: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [2]:
##Checking Google cloud project authentication
datasets = list(client.list_datasets())
project = client.project
print (project)

mining-clinical-decisions


In [3]:
##List Datasets and Tables in project
if datasets:
    print('Datasets in project {}:'.format(project))
    for dataset in datasets:  # API request(s)
        print('\t{}'.format(dataset.dataset_id))
else:
    print('{} project does not contain any datasets.'.format(project))

Datasets in project mining-clinical-decisions:
	ahrq_ccsr
	alert_2019
	clinical_inpatient
	clinical_item2018
	conor_db
	datalake_47618
	datalake_47618_sample
	heme_stamp
	imon_db
	lpch
	shc_test
	starr_datalake2018
	stride_2008_2017
	test_dataset


In [4]:
query = """
SELECT jc_uid, pat_enc_csn_id_coded, effective_time_jittered_utc
FROM starr_datalake2018.adt
WHERE EXTRACT(YEAR from effective_time_jittered_utc) = 2017
ORDER BY jc_uid, effective_time_jittered_utc
"""
query_job =client.query(query)
df=query_job.to_dataframe()


In [ ]:
df.head()

In [14]:
def count_overlapping_encounter_ids():
    counter = 0
    encounter_ids = set()
    encounters = []
    encs = df['pat_enc_csn_id_coded'].values
    temp_encounter_id = encs[0]
    for i in range(len(df)):
        if encs[i] == temp_encounter_id:
            pass
        else:
            if temp_encounter_id in encounter_ids:
                counter += 1  
                encounters.append(temp_encounter_id)
            encounter_ids.add(temp_encounter_id)
            temp_encounter_id = encs[i]
            
    return counter, encounters
        
counter, encs = count_overlapping_encounter_ids()     

In [15]:
encs[0:5]

[131241011902, 131240956469, 131225009679, 131225009683, 131238826600]

In [ ]:
df=query_job.to_dataframe()

In [ ]:
ids = set()
encounters = df['pat_enc_csn_id_coded'].values
for i in range(len(encounters)):
    